In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
df = pd.read_csv("/run/media/nuclea/Code/DataScience/PracticeNotebooks/titanic/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# converting data->numpy array->pytorch tensor
data_numpy = df.to_numpy()
# data_tensor = torch.Tensor(data_numpy)
data_numpy

array([[1, 0, 3, ..., 7.25, nan, 'S'],
       [2, 1, 1, ..., 71.2833, 'C85', 'C'],
       [3, 1, 3, ..., 7.925, nan, 'S'],
       ...,
       [889, 0, 3, ..., 23.45, nan, 'S'],
       [890, 1, 1, ..., 30.0, 'C148', 'C'],
       [891, 0, 3, ..., 7.75, nan, 'Q']], dtype=object)

In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
len(df['Ticket'].value_counts())

681

In [9]:
# removing unnecessary columns
df = df.drop(['Name', 'Ticket'], axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S


In [4]:
df['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [10]:
# # converting sex to int
for i,sex in enumerate(df['Sex']):
    if sex == 'male' or sex == 1:
        df.loc[i, 'Sex'] = 1
    else:
        df.loc[i, 'Sex'] = 2

In [11]:
df['Sex'].value_counts()

Sex
1    577
2    314
Name: count, dtype: int64

In [12]:
df['Sex'] = df['Sex'].astype(int)

In [13]:
len(df['Cabin'].value_counts())

147

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Cabin_int'] = le.fit_transform(df['Cabin'])
df = df.drop(['Cabin'], axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_int
0,1,0,3,1,22.0,1,0,7.2500,S,147
1,2,1,1,2,38.0,1,0,71.2833,C,81
2,3,1,3,2,26.0,0,0,7.9250,S,147
3,4,1,1,2,35.0,1,0,53.1000,S,55
4,5,0,3,1,35.0,0,0,8.0500,S,147


In [15]:
df['Embarked_int'] = le.fit_transform(df['Embarked'])
df = df.drop(['Embarked'], axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_int,Embarked_int
0,1,0,3,1,22.0,1,0,7.2500,147,2
1,2,1,1,2,38.0,1,0,71.2833,81,0
2,3,1,3,2,26.0,0,0,7.9250,147,2
3,4,1,1,2,35.0,1,0,53.1000,55,2
4,5,0,3,1,35.0,0,0,8.0500,147,2


In [16]:
X_df = df.drop('Survived', axis=1)
y_df = df['Survived']

In [17]:
X_df.isna().sum(), X_df.isnull().sum()

(PassengerId       0
 Pclass            0
 Sex               0
 Age             177
 SibSp             0
 Parch             0
 Fare              0
 Cabin_int         0
 Embarked_int      0
 dtype: int64,
 PassengerId       0
 Pclass            0
 Sex               0
 Age             177
 SibSp             0
 Parch             0
 Fare              0
 Cabin_int         0
 Embarked_int      0
 dtype: int64)

In [18]:
df['Age'] = df['Age'].fillna(df['Age'].mean()) 

In [19]:
X_df.isna().sum()

PassengerId       0
Pclass            0
Sex               0
Age             177
SibSp             0
Parch             0
Fare              0
Cabin_int         0
Embarked_int      0
dtype: int64

In [20]:
# converting data->numpy array->tf tensor
X_numpy = X_df.to_numpy()
y_numpy = y_df.to_numpy()
X = torch.tensor(X_numpy)
y = torch.tensor(y_numpy)
X.shape, y.shape

(torch.Size([891, 9]), torch.Size([891]))

In [21]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(9, 300, dtype=torch.float64)
        self.fc2 = nn.Linear(300, 100, dtype=torch.float64)
        self.fc3 = nn.Linear(100, 1, dtype=torch.float64)
    def forward(self, X):
        X = torch.relu(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = torch.sigmoid(self.fc3(X))
        return X

In [22]:
model = Model()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [23]:
losses = []
X = X.long()
for i in range(200000):
    model.train()

    optimizer.zero_grad()
    y_pred = torch.sigmoid(model(X)).squeeze()

    print(y_pred)
    loss = loss_fn(y_pred, y.long())
    loss.backward()
    optimizer.step()

    losses.append(loss)
    if i%10000 == 0:
        print(f"loss[{i}] = {loss}")

RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Double

In [24]:
X.dtype

torch.int64

In [34]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(9, 300, dtype=torch.float64)
        self.fc2 = nn.Linear(300, 100, dtype=torch.float64)
        self.fc3 = nn.Linear(100, 1, dtype=torch.float64)
    def forward(self, X):
        X = torch.relu(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = torch.sigmoid(self.fc3(X))
        return X

model = Model()
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

losses = []
nos = []
X = X.to(torch.float64)
y = y.to(torch.float64)
for i in range(200000):
    model.train()

    optimizer.zero_grad()
    y_pred = model(X).squeeze()

    print(y_pred)
    loss = loss_fn(y_pred, y)
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

    losses.append(loss.item())
    nos.append(i)
    if i%10000 == 0:
        print(f"loss[{i}] = {loss.item()}")
    break

tensor([0.9777, 0.9972, 0.9773, 0.9826, 0.9796, 1.0000, 0.9990, 0.9948, 0.9850,
        0.9978, 0.9917, 0.9330, 0.9830, 0.9974, 0.9814, 0.9919, 0.9973, 1.0000,
        0.9930, 1.0000, 0.9970, 0.9779, 0.9853, 0.9079, 0.9946, 0.9977, 1.0000,
        1.0000, 1.0000, 1.0000, 0.9974, 1.0000, 1.0000, 0.9903, 0.9999, 0.9996,
        1.0000, 0.9865, 0.9936, 0.9889, 0.9882, 0.9953, 1.0000, 0.9990, 0.9846,
        1.0000, 1.0000, 1.0000, 1.0000, 0.9927, 0.9987, 0.9848, 0.9990, 0.9970,
        0.9884, 1.0000, 0.9882, 0.9877, 0.9965, 0.9991, 0.9869, 0.9956, 0.9984,
        0.9967, 1.0000, 1.0000, 0.9908, 0.9882, 0.9852, 0.9901, 0.9928, 0.9992,
        0.9998, 0.9945, 0.9996, 0.9889, 1.0000, 1.0000, 0.9980, 0.9941, 0.9918,
        0.9928, 1.0000, 0.9995, 0.9923, 0.9953, 0.9989, 1.0000, 1.0000, 0.9929,
        0.9928, 0.9925, 0.9995, 0.9972, 0.9920, 1.0000, 0.9846, 0.9989, 0.9978,
        0.9983, 0.9925, 1.0000, 0.9995, 0.9929, 0.9919, 0.9928, 0.9931, 1.0000,
        0.9924, 1.0000, 0.9951, 0.9963, 